# Adicionando as ferramentas ao modelo

A última etapa é realmente chamarmos essas tools quando o modelo solicitar, criando assim uma aplicação funcional que combina modelo com ferramentas criadas pelo usuário.

## Recriando as tools

Vamos primeiro recriar as tools da última aula.

In [3]:
import requests
import datetime

from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

import wikipedia
wikipedia.set_lang('pt')

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return temp_atual
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI()


tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

## Criando roteamento para rodar ferramenta

Chamamos de roteamento o processo de análise da output de um modelos que possui acesso a ferramntas. Noste processo, precisamos entender qual é o tipo de saída que o modelo está nos fornecendo. Caso seja uma saída já com a resposta final do modelo, podemos retorná-la ao usuário, caso o modelo esteja solicitando a chamado de uma ferramenta, temos que chamar essa ferramenta e mostrar o resultado ao usuário.

### Adicionando OpenAIFunctionsAgentOutputParser

Para auxiliar neste processo, podemos utilizar o OpenAIFunctionsAgentOutputParser. Ele processa a saída de um modelo da OpenAi que possui ferramentas e determina o estado da mensagem devolvida.

In [7]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

O retorno será um AgentAction caso necessite que uma ferramenta seja executada.

In [14]:
resultado = chain.invoke({'input': 'Quem foi Isaac Asimov?'})
resultado

AgentActionMessageLog(tool='busca_wikipedia', tool_input={'query': 'Isaac Asimov'}, log="\nInvoking: `busca_wikipedia` with `{'query': 'Isaac Asimov'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'busca_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 154, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1a81360a-a616-4e49-a4f9-727a23afeef7-0')])

In [15]:
resultado.tool

'busca_wikipedia'

In [16]:
resultado.tool_input

{'query': 'Isaac Asimov'}

E um AgentFinish, caso a mensagem esteja finalizada.

In [12]:
resultado = chain.invoke({'input': 'Olá'})
resultado

AgentFinish(return_values={'output': 'Olá! Como posso te ajudar hoje?'}, log='Olá! Como posso te ajudar hoje?')

In [13]:
resultado.return_values['output']

'Olá! Como posso te ajudar hoje?'

### Rodando as ferramentas

Podemos criar uma função simples de roteamento para lidar com os dois estados possíveis:

In [17]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)
    

In [18]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [19]:
chain.invoke({'input': 'Olá'})

'Olá! Como posso ajudar você hoje?'

In [20]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.:Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico norte-americano, nascido na Rússia, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA mai